<img src="https://upload.wikimedia.org/wikipedia/commons/4/47/Logo_UTFSM.png" width="200" alt="utfsm-logo" align="left"/>

# MAT281
### Aplicaciones de la Matemática en la Ingeniería

## Módulo 04
## Laboratorio Clase 04: Métricas y selección de modelos

### Instrucciones


* Completa tus datos personales (nombre y rol USM) en siguiente celda.
* La escala es de 0 a 4 considerando solo valores enteros.
* Debes _pushear_ tus cambios a tu repositorio personal del curso.
* Como respaldo, debes enviar un archivo .zip con el siguiente formato `mXX_cYY_lab_apellido_nombre.zip` a alonso.ogueda@gmail.com, debe contener todo lo necesario para que se ejecute correctamente cada celda, ya sea datos, imágenes, scripts, etc.
* Se evaluará:
    - Soluciones
    - Código
    - Que Binder esté bien configurado.
    - Al presionar  `Kernel -> Restart Kernel and Run All Cells` deben ejecutarse todas las celdas sin error.

__Nombre__: Bernardo Recabarren

__Rol__:201610004-0

En este laboratorio utilizaremos el conjunto de datos _Abolone_. 

**Recuerdo**

La base de datos contiene mediciones a 4177 abalones, donde las mediciones posibles son sexo ($S$), peso entero $W_1$, peso sin concha $W_2$, peso de visceras $W_3$, peso de concha  $W_4$, largo ($L$), diametro $D$, altura $H$, y el número de anillos $A$. 

In [1]:
import pandas as pd
import numpy as np

In [2]:
abalone = pd.read_csv(
    "data/abalone.data",
    header=None,
    names=["sex", "length", "diameter", "height", "whole_weight", "shucked_weight", "viscera_weight", "shell_weight", "rings"]
)

abalone_data = (
    abalone.assign(sex=lambda x: x["sex"].map({"M": 1, "I": 0, "F": -1}))
    .loc[lambda x: x.drop(columns="sex").gt(0).all(axis=1)]
    .astype(np.float)
)
abalone_data.head()

,sex,length,diameter,height,whole_weight,shucked_weight,viscera_weight,shell_weight,rings
0,1.0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15.0
1,1.0,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7.0
2,-1.0,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9.0
3,1.0,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10.0
4,0.0,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7.0


#### Modelo A
Consideramos 9 parámetros, llamados $\alpha_i$, para el siguiente modelo:
$$ \log(A) = \alpha_0 +  \alpha_1 W_1 + \alpha_2 W_2 +\alpha_3 W_3 +\alpha_4 W_4 + \alpha_5 S + \alpha_6 \log L + \alpha_7 \log D+  \alpha_8 \log H$$

In [3]:
def train_model_A(data):
    y = np.log(data.loc[:, "rings"].values.ravel())
    X = (
        data.assign(
            intercept=1.,
            length=lambda x: x["length"].apply(np.log),
            diameter=lambda x: x["diameter"].apply(np.log),
            height=lambda x: x["height"].apply(np.log),
        )
        .loc[: , ["intercept", "whole_weight", "shucked_weight", "viscera_weight", "shell_weight", "sex", "length", "diameter", "height"]]
        .values
    )
    coeffs = np.linalg.lstsq(X, y, rcond=None)[0]
    return coeffs

def test_model_A(data, coeffs):
    X = (
        data.assign(
            intercept=1.,
            length=lambda x: x["length"].apply(np.log),
            diameter=lambda x: x["diameter"].apply(np.log),
            height=lambda x: x["height"].apply(np.log),
        )
        .loc[: , ["intercept", "whole_weight", "shucked_weight", "viscera_weight", "shell_weight", "sex", "length", "diameter", "height"]]
        .values
    )
    ln_anillos = np.dot(X, coeffs)
    return np.exp(ln_anillos)

#### Modelo B
Consideramos 6 parámetros, llamados $\beta_i$, para el siguiente modelo:
$$ \log(A) = \beta_0 + \beta_1 W_1 + \beta_2 W_2 +\beta_3 W_3 +\beta W_4 + \beta_5 \log( L  D H ) $$

In [4]:
def train_model_B(data):
    y = np.log(data.loc[:, "rings"].values.ravel())
    X = (
        data.assign(
            intercept=1.,
            ldh=lambda x: (x["length"] * x["diameter"] * x["height"]).apply(np.log),
        )
        .loc[: , ["intercept", "whole_weight", "shucked_weight", "viscera_weight", "shell_weight", "ldh"]]
        .values
    )
    coeffs = np.linalg.lstsq(X, y, rcond=None)[0]
    return coeffs

def test_model_B(data, coeffs):
    X = (
        data.assign(
            intercept=1.,
            ldh=lambda x: (x["length"] * x["diameter"] * x["height"]).apply(np.log),
        )
        .loc[: , ["intercept", "whole_weight", "shucked_weight", "viscera_weight", "shell_weight", "ldh"]]
        .values
    )
    ln_anillos = np.dot(X, coeffs)
    return np.exp(ln_anillos)

#### Modelo C
Consideramos 12 parámetros, llamados $\theta_i^{k}$, con $k \in \{M, F, I\}$, para el siguiente modelo:

Si $S=male$:
$$ \log(A) = \theta_0^M + \theta_1^M W_2  + \theta_2^M W_4 + \theta_3^M \log( L  D H ) $$

Si $S=female$
$$ \log(A) = \theta_0^F + \theta_1^F W_2  + \theta_2^F W_4 + \theta_3^F \log( L  D H ) $$

Si $S=indefined$
$$ \log(A) = \theta_0^I + \theta_1^I W_2  + \theta_2^I W_4 + \theta_3^I \log( L  D H ) $$

In [5]:
def train_model_C(data):
    df = (
        data.assign(
            intercept=1.,
            ldh=lambda x: (x["length"] * x["diameter"] * x["height"]).apply(np.log),
        )
        .loc[: , ["intercept", "shucked_weight", "shell_weight", "ldh", "sex", "rings"]]
    )
    coeffs_dict = {}
    for sex, df_sex in df.groupby("sex"):
        X = df_sex.drop(columns=["sex", "rings"])
        y = np.log(df_sex["rings"].values.ravel())
        coeffs_dict[sex] = np.linalg.lstsq(X, y, rcond=None)[0]
    return coeffs_dict

def test_model_C(data, coeffs_dict):
    df = (
        data.assign(
            intercept=1.,
            ldh=lambda x: (x["length"] * x["diameter"] * x["height"]).apply(np.log),
        )
        .loc[: , ["intercept", "shucked_weight", "shell_weight", "ldh", "sex", "rings"]]
    )
    pred_dict = {}
    for sex, df_sex in df.groupby("sex"):
        X = df_sex.drop(columns=["sex", "rings"])
        ln_anillos = np.dot(X, coeffs_dict[sex])
        pred_dict[sex] = np.exp(ln_anillos)
    return pred_dict

### 1. Split Data (1 pto)

Crea dos dataframes, uno de entrenamiento (80% de los datos) y otro de test (20% restante de los datos) a partir de `abalone_data`.

_Hint:_ `sklearn.model_selection.train_test_split` funciona con dataframes!

In [6]:
from sklearn.model_selection import train_test_split

abalone_train, abalone_test = train_test_split(abalone_data,test_size=0.2,random_state=42)
abalone_train.head()

SyntaxError: positional argument follows keyword argument (<ipython-input-6-a9475ce8aae7>, line 3)

### 2. Entrenamiento (1 pto)

Utilice las funciones de entrenamiento definidas más arriba con tal de obtener los coeficientes para los datos de entrenamiento. Recuerde que para el modelo C se retorna un diccionario donde la llave corresponde a la columna `sex`.

In [ ]:
coeffs_A = train_model_A(abalone_train)
coeffs_B = train_model_B(abalone_train)
coeffs_C = train_model_C(abalone_train)

### 3. Predicción (1 pto)

Con los coeficientes de los modelos realize la predicción utilizando el conjunto de test. El resultado debe ser un array de shape `(835, )` por lo que debes concatenar los resultados del modelo C. 

**Hint**: Usar `np.concatenate`.

In [ ]:
y_pred_A = test_model_A(abalone_test,coeffs_A)
y_pred_B = test_model_B(abalone_test,coeffs_B)
y_pred_C = np.concatenate([[j for j in test_model_C(abalone_test,coeffs_C)[x]] for x in test_model_C(abalone_test,coeffs_C).keys()])

### 4. Cálculo del error (1 pto)

Se utilizará el Error Cuadrático Medio (MSE) que se define como 

$$\textrm{MSE}(y,\hat{y}) =\dfrac{1}{n}\sum_{t=1}^{n}\left | y_{t}-\hat{y}_{t}\right |^2$$

Defina una la función `MSE` y el vectores `y_test_A`, `y_test_B` e `y_test_C` para luego calcular el error para cada modelo. 

**Ojo:** Nota que al calcular el error cuadrático medio se realiza una resta elemento por elemento, por lo que el orden del vector es importante, en particular para el modelo que separa por `sex`.

In [ ]:
def MSE(y_real, y_pred):
    val= (1/len(y_pred))*sum(abs(y_real-y_pred)**2)
    return val

In [ ]:
y_test_A = abalone_test["rings"]
y_test_B = abalone_test["rings"]
y_test_C = np.concatenate([abalone_test.groupby("sex").get_group(i)["rings"] for i in [-1,0,1]])

In [ ]:
error_A = MSE(y_test_A,y_pred_A)
error_B = MSE(y_test_B,y_pred_B)
error_C = MSE(y_test_C,y_pred_C)

In [ ]:
print(f"Error modelo A: {error_A:.2f}")
print(f"Error modelo B: {error_B:.2f}")
print(f"Error modelo C: {error_C:.2f}")

**¿Cuál es el mejor modelo considerando esta métrica?**

El mejor modelo considerando como métrica el `MSE` es el modelo B, dado que el error es mas pequeño